https://github.com/BAMresearch/NFDI4IngScientificWorkflowRequirements

In [1]:
import os

In [2]:
import subprocess

In [3]:
from pyiron_base import Project

In [4]:
def create_env(env_key):
    subprocess.check_output(
        ["mamba", "env", "create", "-n", env_key, "-f", env_dict[env_key], "-y"],
        universal_newlines=True,
    ).split("\n")

In [5]:
env_dict = {
    "preprocessing": "source/envs/preprocessing.yaml",
    "processing": "source/envs/processing.yaml",
    "postprocessing": "source/envs/postprocessing.yaml"
}

In [6]:
paper_source = "source/paper.tex"
macros_source = "source/macros.tex.template"

In [7]:
pr = Project(".")

# Preprocessing 

In [8]:
env_key = "preprocessing"

In [9]:
create_env(env_key=env_key)

In [10]:
pr_pre = Project("preprocessing")

In [11]:
pr_pre.remove_jobs(recursive=True, silently=True)

## generate mesh

In [12]:
def write_input(input_dict, working_directory):
    script_name = os.path.join(working_directory, "gmsh.sh")
    with open(script_name, "w") as f:
        f.writelines("gmsh -2 -setnumber domain_size " + str(input_dict["domain_size"]) + " unit_square.geo -o square.msh")
    os.chmod(script_name, 0o744)

In [13]:
def collect_output_empty(working_directory):
    return {}

In [14]:
pr.create_job_class(
    class_name="GmshJob",
    write_input_funct=write_input,
    collect_output_funct=collect_output_empty,
    default_input_dict={"domain_size": 2.0},
    executable_str="./gmsh.sh",
)

In [15]:
job_gmsh = pr_pre.create.job.GmshJob(job_name="gmsh")

In [16]:
job_gmsh.server.conda_environment_path = os.path.join("/", "srv", "conda", "envs", env_key)

In [17]:
job_gmsh.restart_file_list.append("source/unit_square.geo")

In [18]:
job_gmsh.run()

The job gmsh was saved and received the ID: 1


## convert to xdmf

In [19]:
def write_input_empty(input_dict, working_directory):
    pass

In [20]:
def collect_output_empty(working_directory):
    return {}

In [21]:
pr.create_job_class(
    class_name="MeshioJob",
    write_input_funct=write_input_empty,
    collect_output_funct=collect_output_empty,
    default_input_dict={"empty": 0},
    executable_str="meshio convert square.msh square.xdmf",
)

In [22]:
job_meshio = pr_pre.create.job.MeshioJob(job_name="meshio")

In [23]:
job_meshio.server.conda_environment_path = os.path.join("/", "srv", "conda", "envs", env_key)

In [24]:
job_meshio.restart_file_list.append(os.path.join(job_gmsh.working_directory, "square.msh"))

In [25]:
job_meshio.run()

The job meshio was saved and received the ID: 2


# Processing

In [26]:
env_key = "processing"

In [27]:
create_env(env_key=env_key)

In [28]:
pr_main = Project("processing")

In [29]:
pr_main.remove_jobs(recursive=True, silently=True)

## poisson

In [30]:
def collect_output(working_directory):
    with open(os.path.join(working_directory, "numdofs.txt"), "r") as f:
        return {"numdofs": int(f.read())}

In [31]:
pr.create_job_class(
    class_name="PoissonJob",
    write_input_funct=write_input_empty,
    collect_output_funct=collect_output,
    default_input_dict={"empty": 0},
    executable_str="python poisson.py --mesh square.xdmf --degree 2 --outputfile poisson.pvd --num-dofs numdofs.txt",
)

In [32]:
job_poisson = pr_main.create.job.PoissonJob(job_name="poisson")

In [33]:
job_poisson.server.conda_environment_path = os.path.join("/", "srv", "conda", "envs", env_key)

In [34]:
job_poisson.restart_file_list.append(os.path.join(job_meshio.working_directory, "square.xdmf"))

In [35]:
job_poisson.restart_file_list.append(os.path.join(job_meshio.working_directory, "square.h5"))

In [36]:
job_poisson.restart_file_list.append(os.path.abspath("source/poisson.py"))

In [37]:
job_poisson.run()

The job poisson was saved and received the ID: 3


In [38]:
job_poisson.output["numdofs"]

357

# Postprocessing

In [39]:
env_key = "postprocessing"

In [40]:
create_env(env_key=env_key)

In [41]:
pr_post = Project("postprocessing")

In [42]:
pr_post.remove_jobs(recursive=True, silently=True)

## plot over line

In [43]:
pr.create_job_class(
    class_name="PvbatchJob",
    write_input_funct=write_input_empty,
    collect_output_funct=collect_output_empty,
    default_input_dict={"empty": 0},
    executable_str="pvbatch postprocessing.py poisson.pvd plotoverline.csv",
)

In [44]:
job_pvbatch = pr_post.create.job.PvbatchJob(job_name="pvbatch")

In [45]:
job_pvbatch.server.conda_environment_path = os.path.join("/", "srv", "conda", "envs", env_key)

In [46]:
job_pvbatch.restart_file_list.append(os.path.join(job_poisson.working_directory, "poisson.pvd"))

In [47]:
job_pvbatch.restart_file_list.append(os.path.join(job_poisson.working_directory, "poisson000000.vtu"))

In [48]:
job_pvbatch.restart_file_list.append(os.path.abspath("source/postprocessing.py"))

In [49]:
job_pvbatch.run()

The job pvbatch was saved and received the ID: 4


## substitute macros

In [50]:
def write_input(input_dict, working_directory):
    script_name = os.path.join(working_directory, "macros.sh")
    with open(script_name, "w") as f:
        f.writelines("python prepare_paper_macros.py --macro-template-file macros.tex.template --plot-data-path plotoverline.csv --domain-size " + str(input_dict["domain_size"]) + " --num-dofs " + str(input_dict["numdofs"]) + " --output-macro-file macros.tex")
    os.chmod(script_name, 0o744)

In [51]:
pr.create_job_class(
    class_name="MacrosJob",
    write_input_funct=write_input,
    collect_output_funct=collect_output_empty,
    default_input_dict={"domain_size": 2.0, "numdofs": 100},
    executable_str="./macros.sh",
)

In [52]:
job_macros = pr_post.create.job.MacrosJob(job_name="macros")

In [53]:
job_macros.input["numdofs"] = job_poisson.output["numdofs"]

In [54]:
job_macros.server.conda_environment_path = os.path.join("/", "srv", "conda", "envs", env_key)

In [55]:
job_macros.restart_file_list.append(os.path.abspath("source/macros.tex.template"))

In [56]:
job_macros.restart_file_list.append(os.path.abspath("source/prepare_paper_macros.py"))

In [57]:
job_macros.restart_file_list.append(os.path.join(job_pvbatch.working_directory, "plotoverline.csv"))

In [58]:
job_macros.run()

The job macros was saved and received the ID: 5


## compile paper

In [59]:
pr.create_job_class(
    class_name="TectonicJob",
    write_input_funct=write_input_empty,
    collect_output_funct=collect_output_empty,
    default_input_dict={"empty": 0},
    executable_str="tectonic paper.tex",
)

In [60]:
job_tectonic = pr_post.create.job.TectonicJob(job_name="tectonic")

In [61]:
job_tectonic.server.conda_environment_path = os.path.join("/", "srv", "conda", "envs", env_key)

In [62]:
job_tectonic.restart_file_list.append(os.path.abspath("source/paper.tex"))

In [63]:
job_tectonic.restart_file_list.append(os.path.join(job_macros.working_directory, "macros.tex"))

In [64]:
job_tectonic.restart_file_list.append(os.path.join(job_pvbatch.working_directory, "plotoverline.csv"))

In [65]:
job_tectonic.run()

The job tectonic was saved and received the ID: 6


# Overview


In [66]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,1,finished,None,gmsh,/gmsh,None,/home/jovyan/preprocessing/,2024-01-30 21:27:45.128069,2024-01-30 21:27:46.633350,1.0,pyiron@jupyter-jan-2djanssen-2dnfdi4ing-5fpyiron-5fbase-2dpoykt4ps#1,ExecutableContainerJob,0.4,None,None
1,2,finished,None,meshio,/meshio,None,/home/jovyan/preprocessing/,2024-01-30 21:27:46.912108,2024-01-30 21:27:48.338132,1.0,pyiron@jupyter-jan-2djanssen-2dnfdi4ing-5fpyiron-5fbase-2dpoykt4ps#1,ExecutableContainerJob,0.4,None,None
2,3,finished,None,poisson,/poisson,None,/home/jovyan/processing/,2024-01-30 21:28:35.532079,2024-01-30 21:28:49.419680,13.0,pyiron@jupyter-jan-2djanssen-2dnfdi4ing-5fpyiron-5fbase-2dpoykt4ps#1,ExecutableContainerJob,0.4,None,None
3,4,finished,None,pvbatch,/pvbatch,None,/home/jovyan/postprocessing/,2024-01-30 21:29:35.059995,2024-01-30 21:29:37.483323,2.0,pyiron@jupyter-jan-2djanssen-2dnfdi4ing-5fpyiron-5fbase-2dpoykt4ps#1,ExecutableContainerJob,0.4,None,None
4,5,finished,None,macros,/macros,None,/home/jovyan/postprocessing/,2024-01-30 21:29:37.785818,2024-01-30 21:29:38.683928,0.0,pyiron@jupyter-jan-2djanssen-2dnfdi4ing-5fpyiron-5fbase-2dpoykt4ps#1,ExecutableContainerJob,0.4,None,None
5,6,finished,None,tectonic,/tectonic,None,/home/jovyan/postprocessing/,2024-01-30 21:29:38.965947,2024-01-30 21:29:42.736432,3.0,pyiron@jupyter-jan-2djanssen-2dnfdi4ing-5fpyiron-5fbase-2dpoykt4ps#1,ExecutableContainerJob,0.4,None,None
